In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px
import random

from manofunkcijos import *

import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver # naršyklės kontroleris
from selenium.webdriver.chrome.options import Options #Naršyklės
# pip install beautifulsoup4 selenium
import time
import nordvpn_switcher as ns

In [54]:
# Informacijos gavimas iš autoplius.lt
# Užduotys
# Naudodamiesi selenium, beautifulsoup bibliotekomis bei pasirinkta naršykle, nuskaitykite
# autoplius.lt skelbimų informaciją.
# Nuskaityta informacija turi būti saugoma jūsų susikurtoje SQLite duomenų bazėje (DB).

# Jūsų skriptas turi surinkti ir į DB surašyti visą reikiamą infromaciją, kuri būtų reikalinga toli-
# mesnei analizei. Vizualizacijoms/analizei duomenis toliau turite imti iš savo užpildytos SQLi-
# te DB.

# Klausimai:

# Įvertinkite automobilių kainos priklausomybes nuo ridos, amžiaus, variklio galios, kėbulo ti-
# po, pavarų dėžės tipo. Įvertinkite, ar kainos dydžiui turi įtaką pardavėjo miestas/rajonas? Ko-
# kios šalys vyrauja kaip pirmos registracijos šalis? Ar dėl to yra kokia nors įtaka kainoms?

# Kiek skelbimų nurodo galiojančią TA? Ar TA turėjimas ir TA galiojimo trukmė turi įtaką
# kainų lygiui?
# Kokie 10 gamintojų buvo populiariausi? Pateikite šių gamintojų automobiliams kainos

# kitimo trendus nuo ridos, amžiaus, variklio galios. Kurio gamintojo automobiliai pigo spar-
# čiausiai, o kuris gamintojas(-ai) ilgiausiai išlaikydavo savo vertę (kaina nuo ridos/amžiaus)?

# Kuris gamintojas/gamintojai turėjo didžiausią kainų išsibarstymą? Suraskite 10 retesnių au-
# tomobilių gamintojų (5 < x < 15 vnt).

# Pateikite 10 populiariausių gamintojų variklio galių (kW), ridų, amžiaus, rinkoje esančių
# modelių kiekio vienetais pasiskirstymą (vidurkiai, medianos, kvartilės).

# Ar yra priklausomybė tarp kainų ir elektromobilių akumuliatoriaus talpos (kWh), el. varik-
# lio galios (kW)? Kaip sparčiai pigo elektromobiliai? Ar buvo gamintojų, kurių elektromobiliai

# ilgai išlaikydavo vertę (pigo lėčiausiai)?
# Ar elektromobilių kainos nuo ridos, amžiaus priklauso (kito) labiau, nei VDV automobilių?
# Kiek skelbimuose buvo automobilių su defektais, be defektų? Kokie gamintojai vyravo
# skelbimuose, nurodančiuose defektus?
# Visiems klausimams parinkite ir pateikite vizualizacijas, leidžiančias atsakyti į klausimus.
# Vizualizacijas pateikti streamlit aplinkoje.

In [ ]:
auto_skelbimai = []
# ns.initialize_VPN(save=1,area_input=['complete rotation'])


while True:
    try:
        opcijos = Options()
        opcijos.add_argument('--incognito')
        driver = webdriver.Chrome(options=opcijos)
        url = 'https://autoplius.lt/'
        driver.get(url)
        time.sleep(3)
        source = driver.page_source
        bs = BeautifulSoup(source, 'html.parser')
        markes = bs.find_all('div', {'class':'dropdown-option js-option'})
        for m in markes:
            try:
                puslapiai = int(200) if int(np.ceil(int(m['data-badge'])/20))>int(200) else int(np.ceil(int(m['data-badge'])/20))
                print(puslapiai)
                markes_id = int(m['data-value'])
            except:
                None
            opcijos = Options()
            opcijos.add_argument('--incognito')
            driver = webdriver.Chrome(options=opcijos)
            # maxp = int(check_if_value_exists_or_max('web_scrap.db','autopliuslt','Page'))
            # maxp = 105
            for p in range(puslapiai):
                # ns.rotate_VPN()
                # time.sleep(60)
                page = p+1
                print(markes_id,page)
                try:
                    url = f'https://autoplius.lt/skelbimai/naudoti-automobiliai?make_id={markes_id}&page_nr={page}'
                    driver.get(url)
                
                    time.sleep(3)
                    source = driver.page_source
                    bs = BeautifulSoup(source, 'html.parser')
                    straipsniai = bs.find('div', {'class':'auto-lists lt'}).find_all('a')
                

                    for s in straipsniai:
                        if s is not None:
                            url = s['href']
                            if not check_if_value_exists_or_max('web_scrap.db','autopliuslt2','URL',url):
                                driver.get(url)
                                
                                source = driver.page_source
                                bs = BeautifulSoup(source, 'html.parser')
                                
                                skelbimas = bs.find_all('div', {'class':'parameter-row'})
                                marke = bs.find_all('li', {'class':'crumb'})[2].text.strip()
                                modelis = bs.find_all('li', {'class':'crumb'})[3].text.strip()
                                kaina = bs.find('div', {'class':'price'}).text.strip().split('\n')[0].replace(' ','')[:-1]
                                miestas = bs.find_all('span', {'class':'seller-contact-location'})[0].text.strip()
                                skelb = {}
                                skelb['Markė']=marke
                                skelb['Modelis']=modelis
                                skelb['Kaina']=kaina
                                skelb['Miestas']=miestas
                                for s in skelbimas:
                                    x = s.find_all('div', {'class':'parameter-value'})
                                    p = s.find_all('div', {'class':'parameter-label'})
                                    for i, z in zip(x,p):
                                        skelb[z.text.strip()]=i.text.strip()
                                skelb['URL'] = url
                                skelb['Page'] = page
                                auto_skelbimai.append(skelb)
                                insert_into_db(skelb,'web_scrap.db','autopliuslt2')
                except:
                    None

        driver.close()
    except Exception as e:
        print(e)
        driver.close()
    time.sleep(random.randint(5,100))


1
106 1
1
33934 1
1
104 1
1
33416 1
2
108 1
108 2
7
103 1
103 2
103 3
103 4
103 5
103 6
103 7
1
112 1
1
100 1
199
99 1
99 2
99 3
99 4
99 5
99 6
99 7
99 8
99 9
99 10
99 11
99 12
99 13
99 14
99 15
99 16
99 17
99 18
99 19
99 20
99 21
99 22
99 23
99 24
99 25
99 26
99 27
99 28
99 29
99 30
99 31
99 32
99 33
99 34
99 35
99 36
99 37
99 38
99 39
99 40
99 41
99 42
99 43
99 44
99 45
99 46
99 47
99 48
99 49
99 50
99 51
99 52
99 53
99 54
99 55
99 56
99 57
99 58
99 59
99 60
99 61
99 62
99 63
99 64
99 65
99 66
99 67
99 68
99 69
99 70
99 71
99 72
99 73
99 74
99 75
99 76
99 77
99 78
99 79
99 80
99 81
99 82
99 83
99 84
99 85
99 86
99 87
99 88
99 89
99 90
99 91
99 92
99 93
99 94
99 95
99 96
99 97
99 98
99 99
99 100
99 101
99 102
99 103
99 104
99 105
99 106
99 107
99 108
99 109
99 110
99 111
99 112
99 113
99 114
99 115
99 116
99 117
99 118
99 119
99 120
99 121
99 122
99 123
99 124
99 125
99 126
99 127
99 128
99 129
99 130
99 131
99 132
99 133
99 134
99 135
99 136
99 137
99 138
99 139
99 140
99 141
99 142


In [1]:
for p in range(2):
    print(p+1)

1
2


In [5]:

db_path = 'web_scrap.db'
query = 'SELECT * FROM autopliuslt where Miestas = NULL;' 

dfs = query_to_dataframe(db_path, query)


In [6]:
urls= list(df['URL'])

In [ ]:
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = f'https://autoplius.lt/'
source = driver.page_source
bs = BeautifulSoup(source, 'html.parser')
driver.get(url)
time.sleep(5)
for url in urls:
    try:
        data = {}
        opcijos = Options()
        opcijos.add_argument('--incognito')
        driver = webdriver.Chrome(options=opcijos)
        driver.get(url)
        
        source = driver.page_source
        # driver.close()
        bs = BeautifulSoup(source, 'html.parser')
        miestas = bs.find_all('span', {'class':'seller-contact-location'})[0].text.strip()
        data['Miestas'] = miestas
        update_row_in_db('web_scrap.db', 'autopliuslt', data, 'URL', url)
    except:
        driver.close()
driver.close()

Atnaujinta eilutė, kur URL = 'https://autoplius.lt/skelbimai/bmw-428-gran-coupe-2-0-l-kupe-coupe-2015-benzinas-26837939.html': nustatyta Miestas = 'Klaipėda, Lietuva'
Atnaujinta eilutė, kur URL = 'https://autoplius.lt/skelbimai/bmw-530-2-0-l-sedanas-2019-benzinas-26808537.html': nustatyta Miestas = 'Klaipėda, Lietuva'
Atnaujinta eilutė, kur URL = 'https://autoplius.lt/skelbimai/bmw-530-2-0-l-sedanas-2021-benzinas-26749453.html': nustatyta Miestas = 'Klaipėda, Lietuva'
Atnaujinta eilutė, kur URL = 'https://autoplius.lt/skelbimai/fiat-500x-2-4-l-visureigis-krosoveris-2016-benzinas-dujos-26862175.html': nustatyta Miestas = 'Kaunas, Lietuva'
Atnaujinta eilutė, kur URL = 'https://autoplius.lt/skelbimai/bmw-320-gran-turismo-2-0-l-hecbekas-2016-dyzelinas-26830873.html': nustatyta Miestas = 'Kėdainiai, Lietuva'
Atnaujinta eilutė, kur URL = 'https://autoplius.lt/skelbimai/mazda-cx-5-2-0-l-visureigis-krosoveris-2017-benzinas-26740367.html': nustatyta Miestas = 'Klaipėda, Lietuva'
Atnaujinta eilu